# Clarkston Consulting Interview Assignment
This notebook analyzes the CustomerExtract.csv file and provides the following:
<ol>
<li>Reads in the CustomerExtract.csv</li>
<li>Completes an initial profile of the data and performs data quality checks that you think are needed e.g. Are there any fields in the extract that are populated with data that are not flagged in the spec (Field Utilized in LEGACY System)<ol><li>Quality checks should be visible in the notebook and output for traceability</li></ol></li>
<li>Cleanup any data quality issues, such as special characters in the Name 1 field</li>
<li>Perform the transformations included in the spec</li>
<li>Output a flat file named CustomerLoad.csv</li>
</ol>

## Imports

In [580]:
import pandas as pd
import numpy as np
import regex as re

## Prep Spec Document Into Dataframe For Dataset Analysis

In [581]:
# Specify Usable Column Names For Later
customer_spec_columns = [
    'field_label',
    'utilized_in_legacy_system',
    'utilized_in_new_system',
    'table_name',
    'field_name',
    'field_length',
    'field_datatype',
    'field_used',
    'field_description',
    'required_in_legacy_system',
    'required_in_new_system',
    'fit_or_gap',
    'etl_logic'
]

# Read Excel Doc into DF
customer_spec_df = pd.read_excel(
    'Dataset/CustomerSpec.xlsx',
    header=None, # Replace Header With Spec Columns Dictionary
    skipfooter=0,
    skiprows=1,
    sheet_name='CustomerSpec',
    names=customer_spec_columns
)

/Users/tvisser/Library/Mobile Documents/com~apple~CloudDocs/Projects/clarkston_consulting/env/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [582]:
# Replace N/Y With Booleans
boolean_columns = ['utilized_in_legacy_system', 'utilized_in_new_system', 'field_used']
customer_spec_df[boolean_columns] = customer_spec_df[boolean_columns].replace({'Y': True, 'N' :False})

# Add a lookup field to map to the extract
customer_spec_df['extract_field_name'] = customer_spec_df['field_label'].str.lower().replace({
    r'[^A-Z|a-z|0-9]$': '',
    r'[^A-Z|a-z|0-9]': '_'
    }, regex=True)

## Extract Analysis

In [591]:
# Read CSV into Pandas dataframe
customer_extract_df = pd.read_csv(
    'Dataset/CustomerExtract.csv',
    sep=',',
    header=0,
    true_values=['Y'],
    false_values=['N'],
    dtype=object
)

In [592]:
# Drop any unused columns
# used_columns = [col for col in customer_spec_df['field_name'].loc[customer_spec_df['field_used'] == True] if col in customer_extract_df.columns]
cols_to_drop = [col for col in customer_spec_df['field_name'].loc[customer_spec_df['field_used'] == False] if col in customer_extract_df.columns]

customer_extract_df.drop(columns=cols_to_drop, inplace=True)


In [593]:
# Rename Extract Columns From Spec
customer_extract_columns = {}

for index, field_name, field_label in customer_spec_df[['field_name', 'field_label']].itertuples():
    # Remove trailing special characters
    field_label = re.sub(r'[^A-Z|a-z|0-9]$', '', field_label)
    # Replace remaining special characters with underscores
    field_label = re.sub(r'[^A-Z|a-z|0-9]', '_', field_label)
    # Lower Case field label
    field_label = field_label.lower()
    if field_label not in customer_extract_columns.values() and not customer_extract_columns.get(field_name):
        customer_extract_columns.update({field_name: field_label})

customer_extract_df.rename(columns=customer_extract_columns, inplace=True)

In [594]:
# Add Missing Required Fields Using Selection Logic

# company_code
customer_extract_df['company_code'] = 'G100'

# sales_organization
customer_extract_df['sales_organization'] = 'G100'

# distrubtion_channel
customer_extract_df['distribution_channel'] = '20'

# division
customer_extract_df['division'] = '10'

In [595]:
# Update Data Types Using Spec
data_type_map = {
    'CHAR': "str",
    'NUMC': "numeric",
    'DATS': "datetime",
    'LANG': "str",
    'CURR': "numeric",
    'CUKY': "numeric",
    'TIMS': "datetime",
    'QUAN': "numeric",
    'UNIT': "numeric",
}

for index, field_label, field_datatype in customer_spec_df[['extract_field_name', 'field_datatype']].itertuples():
    if field_label in customer_extract_df.columns:
        data_type = data_type_map[field_datatype]
        if data_type == "datetime":
            customer_extract_df[field_label] = pd.to_datetime(customer_extract_df[field_label], errors='coerce')
        elif data_type == "numeric":
            customer_extract_df[field_label] = pd.to_numeric(customer_extract_df[field_label], errors='coerce')
        elif data_type == "str":
            customer_extract_df[field_label] = customer_extract_df[field_label].astype(str)

# Replace Nulls With Proper Nulls
customer_extract_df.replace("nan", None, inplace=True)

/var/folders/f7/p92k225d4t57x80m5zl_8jjh0000gn/T/ipykernel_57692/3328011843.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  customer_extract_df[field_label] = pd.to_datetime(customer_extract_df[field_label], errors='coerce')


In [596]:
# Find any required columns in the extract missing from the spec
required_fields = customer_spec_df['extract_field_name'].loc[customer_spec_df['required_in_new_system'] == True]

missing_fields = []
for col in required_fields:
    if col not in customer_extract_df.columns:
        missing_fields.append(col)

if missing_fields:
    print("THE FOLLOWING COLUMNS ARE MISSING FROM THE EXTRACT AND ARE REQUIRED IN THE LEGACY SYSTEM")
    [print(fields) for fields in missing_fields]
else:
    print('ALL REQUIRED FIELDS IN NEW SYSTEM EXIST IN EXTRACT')

ALL REQUIRED FIELDS IN NEW SYSTEM EXIST IN EXTRACT


In [597]:
# Find any required columns in the extract missing from the spec
used_fields = customer_spec_df['extract_field_name'].loc[customer_spec_df['field_used'] == True]

missing_fields = []
for col in used_fields:
    if col not in customer_extract_df.columns:
        missing_fields.append(col)

if missing_fields:
    print("THE FOLLOWING COLUMNS ARE MISSING FROM THE EXTRACT AND ARE REQUIRED IN THE LEGACY SYSTEM")
    [print(fields) for fields in missing_fields]
else:
    print('ALL USED FIELDS IN NEW SYSTEM EXIST IN EXTRACT')

THE FOLLOWING COLUMNS ARE MISSING FROM THE EXTRACT AND ARE REQUIRED IN THE LEGACY SYSTEM
account_group


In [599]:
# Check For Truncation
field_data_types = customer_spec_df[['extract_field_name', 'field_datatype', 'field_length']].loc[customer_spec_df['utilized_in_new_system'] == True]

has_truncation = False
for index, field_name, field_datatype, field_length in field_data_types.itertuples():
    # Check for truncation
    if data_type_map[field_datatype] == "str" and field_name in customer_extract_df.columns:
        truncated_records = customer_extract_df[field_name].loc[
            (customer_extract_df[field_name].astype(str).str.len() > field_length) & (customer_extract_df[field_name].isna() == False)
        ].drop_duplicates()

        if not truncated_records.empty:
            print(field_name, 'has', len(truncated_records), 'truncated records. Max length for column is', field_length)
            has_trunction = True

if not has_truncation:
    print('NO TRUNCATED COLUMNS FOUND')


NO TRUNCATED COLUMNS FOUND


In [608]:
[col for col in customer_extract_df.select_dtypes(include='object').columns]

['customer_account_number',
 'country',
 'name_1',
 'name_2',
 'city',
 'postal_code',
 'region',
 'sort_field',
 'street',
 'telephone_1',
 'order_block',
 'created_by',
 'vendor',
 'delivery_block',
 'deletion_flag',
 'name_3',
 'name_4',
 'county_code',
 'city_code',
 'posting_block',
 'language',
 'tax_number_1',
 'tax_number_2',
 'transport_zone',
 'al_payer_in_doc',
 'trading_partner',
 'vat_reg__no',
 'oi_allowed',
 'cust_complexity',
 'edi_credit_mgmt',
 'eft_credit_mgmt',
 'dir_store_dlvry',
 'interface',
 'labels',
 'addenda_info',
 'edi_flags',
 'attribute_10',
 'tax_jur',
 'condition_grp_1',
 'condition_grp_2',
 'condition_grp_3',
 'condition_grp_4',
 'condition_grp_5',
 'tax_number',
 'RIC',
 'LEGALNAT',
 'company_code',
 'sales_organization',
 'distribution_channel',
 'division']

In [609]:
# Find columns with data integrity / formatting needs
string_columns = [customer_extract_df.select_dtypes(include='object').columns]

customer_extract_df[[string_columns]].replace(r'[^A-Z|a-z|0-9]', '', regex=True, inplace=True)

ValueError: 3

In [550]:
# Output file
used_fields = customer_spec_df['extract_field_name'].loc[customer_spec_df['field_used'] == True]

customer_extract_df[used_fields].to_csv('CustomerLoad.csv', index=False)

# My Process

<ul>
<li>Convert the spec into a usable format</li>
<li>Make the extract fields make sense</li>
<li>Identify anything in the extract that will break the new system according to the spec (missing fields, bad types, truncation, etc)</li>
</ul>

# Found Problems

<ul>
    <li>Account Group and Customer Account have ambiguous field names in the extract. There are two fields in the spec with distinct table/column name combos but the extract doesn't specify which is which.</li>
    <li>Duplicate SAP Field Descriptions</li>
    <li>Leading and trailing special characters for the field descriptions. This is only an issue due to converting the field names in the extract to friendly terms</li>
</ul>